# Place a molecule onto a Surface

In [ ]:
from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)

from aiida.orm.querybuilder import QueryBuilder
from aiida.orm.data.structure import StructureData
from aiida.orm.data.base import Int
from aiida.work.workfunction import workfunction

from ase.data import covalent_radii
from ase.neighborlist import NeighborList
import ase.neighborlist

from IPython.display import display, clear_output
import ipywidgets as ipw
import numpy as np
from numpy.linalg import norm
import scipy.stats
import nglview

from apps.surfaces.structure_browser import StructureBrowser

## Step 1: Select a structure from the AiiDA database

In [ ]:
def on_struct_change(c):
    global orig_structure
    orig_structure = None # disable event processing
    s = struct_browser.results.value
    if s:
        atoms = s.get_ase()
        nx, ny = guess_slab_size(atoms)
        nx_slider.value = nx
        ny_slider.value = ny
        nx_slider.max = 3*nx
        ny_slider.max = 3*ny
        orig_structure = s
    update_view()

In [ ]:
struct_browser = StructureBrowser()
struct_browser.results.observe(on_struct_change, names='value')
display(struct_browser)

## Step 2: Select cell size

In [ ]:
def on_nxy_change(c):
    update_view()
    
nx_slider = ipw.IntSlider(description="nx", min=1, continuous_update=False)
nx_slider.observe(on_nxy_change, names='value')
ny_slider = ipw.IntSlider(description="ny", min=1, continuous_update=False)
ny_slider.observe(on_nxy_change, names='value')

display(nx_slider, ny_slider)

In [ ]:
def update_view():

    with info_out:
        clear_output()

        inp_descr.value = ""

        # remove old components
        if hasattr(viewer, "component_0"):
            viewer.component_0.remove_ball_and_stick()
            viewer.component_0.remove_unitcell()
            cid = viewer.component_0.id
            viewer.remove_component(cid)

        if orig_structure:
            nx = nx_slider.value
            ny = ny_slider.value
            inp_descr.value = orig_structure.description + ".slap(%d,%d)"%(nx,ny)
            atoms = prepare_mol_on_slab(orig_structure.get_ase(), nx=nx, ny=ny)

            # add new component
            viewer.add_component(nglview.ASEStructure(atoms)) # adds ball+stick
            viewer.add_unitcell()
            viewer.center_view()

In [ ]:
viewer = nglview.NGLWidget()
info_out = ipw.Output()
display(viewer, info_out)

In [ ]:
# dimensions of slab 
Au_x = 2.973735971
Au_y = 5.150661791

def guess_slab_size(mol):
    cx = np.amax(mol.positions[:,0]) - np.amin(mol.positions[:,0]) + 10
    cy = np.amax(mol.positions[:,1]) - np.amin(mol.positions[:,1]) + 10
    nx = int(round(cx/Au_x))
    ny = int(round(cy/Au_y))
    return nx, ny

def prepare_mol_on_slab(mol, nx, ny):
    # determine cell size
    auz_top = 18.49
    dminau = 2.3
    
    cz = 40
    cx = nx * Au_x
    cy = ny * Au_y
    print("Cell ABC: %f, %f, %f"%(cx, cy, cz))
    mol.cell = (cx,cy,cz)
    mol.pbc = (True,True,True)
    
    # position molecule a bit above gold slab
    mol.center()
    minz = np.amin(mol.positions[:,2])
    dz = (-minz + auz_top + dminau)
    mol.positions[:,2] += dz
    
    # template for gold slab
    au_templ = np.array([(1 ,   1.502273797989   ,  3.091176986694   , 10.434000015259),
                         (1 ,   0.017487669364   ,  0.521309018135   , 10.439200401306),
                         (79,   0.011242070235   ,  3.954530715942   , 11.276800155640),
                         (79,   1.498110055923   ,  1.378893256187   , 11.276800155640),
                         (79,   0.011242070235   ,  2.237301826477   , 13.704799652100),
                         (79,   1.498110055923   ,  4.812939167023   , 13.704799652100),
                         (79,   1.498110055923   ,  3.095984896024   , 16.073392868042),
                         (79,   0.011242070235   ,  0.520347436269   , 16.073581695557),
                         (79,   0.011242070235   ,  3.954530715942   , 18.493516921997),
                         (79,   1.498110055923   ,  1.378893256187   , 18.495817184448) ])
    
    
    # generate gold slab
    au_slab_raw = []
    for i in range(nx):
        for j in range(ny):
            shift = np.array([0, i*Au_x, j*Au_y, 0])
            au_slab_raw.append(au_templ + shift)

    au_slab_raw = np.concatenate(au_slab_raw)
    au_slab = ase.Atoms(numbers=au_slab_raw[:,0], positions=au_slab_raw[:,1:])
    
    mol_on_au = mol + au_slab
    return mol_on_au

## Step 3: Store structure in the AiiDA database

In [ ]:
def on_click_store(b):
    if not orig_structure:
        print("No structure selected.")
        return

    nx = Int(nx_slider.value)
    ny = Int(ny_slider.value)
    s = prepare_mol_on_slab_wf(orig_structure, nx, ny)
    s.description = inp_descr.value
    s.store()
    print("Stored in AiiDA: "+repr(s))

cell_ready = False
inp_descr = ipw.Text(placeholder="Description (optional)")   
btn_store = ipw.Button(description='Store in AiiDA')
btn_store.on_click(on_click_store)
display(ipw.HBox([btn_store, inp_descr]))

In [ ]:
@workfunction
def prepare_mol_on_slab_wf(orig_struct, nx, ny):
    nx, ny = nx.value, ny.value
    orig_atoms = orig_struct.get_ase()
    new_atoms = prepare_mol_on_slab(orig_atoms, nx, ny)
    return StructureData(ase=new_atoms)